In [111]:

import pandas as pd

accounts_df = pd.read_excel("Processed_file\Cleaned_data\cleaned_accounts.xlsx", sheet_name='Sheet1')
deposits_df = pd.read_excel("Processed_file\Cleaned_data\cleaned_deposits.xlsx", sheet_name='Sheet1')
loans_df = pd.read_excel("Processed_file\Cleaned_data\cleaned_loans.xlsx", sheet_name='Sheet1')

# Data Modelling

In [112]:
accounts_df.rename(columns={'amount': 'account_amount'}, inplace=True)
deposits_df.rename(columns={'amount': 'deposit_amount'}, inplace=True)
loans_df.rename(columns={'amount': 'loan_amount'}, inplace=True)


## Creating Dimension Tables

### Customer_Dim

In [113]:
import numpy as np
unique_customers_deposits = deposits_df[['customer', 'customer_type']].drop_duplicates()
unique_customers_loans = loans_df[['customer', 'customer_type']].drop_duplicates()

# Combine the unique customers and remove duplicates
all_unique_customers = pd.concat([unique_customers_deposits, unique_customers_loans]).drop_duplicates()

# Create a DataFrame for unique customers with customer_key
unique_customers_df = all_unique_customers.copy()
unique_customers_df['customer_key'] = range(1, len(unique_customers_df) + 1)

# Display the first few rows of the unique customers DataFrame
# print(unique_customers_df)

### Country_Dimension

In [114]:
unique_countries_deposits = deposits_df['country'].unique()
unique_countries_loans = loans_df['country'].unique()

# Combine the unique countries and remove duplicates
all_unique_countries = np.unique(np.concatenate((unique_countries_deposits, unique_countries_loans)))

# Create a DataFrame for unique countries
unique_countries_df = pd.DataFrame(all_unique_countries, columns=['country'])
unique_countries_df['country_key'] = range(1, len(unique_countries_df) + 1)

# Display the first few rows of the unique countries DataFrame
print(unique_countries_df)

  country  country_key
0      FI            1
1      NO            2
2      SE            3


### Currency_Dimension

In [115]:
unique_currencies_deposits = deposits_df['currency'].unique()
unique_currencies_loans = loans_df['currency'].unique()

# Combine the unique currencies and remove duplicates
all_unique_currencies = np.unique(np.concatenate((unique_currencies_deposits, unique_currencies_loans)))

# Create a DataFrame for unique currencies
unique_currencies_df = pd.DataFrame(all_unique_currencies, columns=['currency'])
unique_currencies_df['currency_key'] = range(1, len(unique_currencies_df) + 1)

# Display the first few rows of the unique currencies DataFrame
print(unique_currencies_df)

  currency  currency_key
0      EUR             1
1      NOK             2
2      SEK             3


### Accounts_Name_Dimension

In [116]:
unique_account_names = pd.DataFrame(accounts_df['account_name'].unique(), columns=['account_name'])
unique_account_names['account_name_key'] = range(1, len(unique_account_names) + 1)

# Display the account name dimension DataFrame
print(unique_account_names)


                    account_name  account_name_key
0  Unsecured Personal Loan - EUR                 1
1              Credit Card - NOK                 2
2        Corporate Leasing - SEK                 3
3  Unsecured Personal Loan - SEK                 4
4                    Fixed - EUR                 5
5                 Floating - EUR                 6
6                    Fixed - SEK                 7
7                 Floating - SEK                 8


### Date_Dimension

1. First we find the minimum and maximum date in data (Can be optimized for scaling) 

2. Create Date Dimension table

In [117]:
# Convert relevant date columns to datetime
accounts_df['reference_date'] = pd.to_datetime(accounts_df['reference_date'])
deposits_df['start_date'] = pd.to_datetime(deposits_df['start_date'])
deposits_df['maturity_date'] = pd.to_datetime(deposits_df['maturity_date'])
deposits_df['reference_date'] = pd.to_datetime(deposits_df['reference_date'])
loans_df['start_date'] = pd.to_datetime(loans_df['start_date'])
loans_df['maturity_date'] = pd.to_datetime(loans_df['maturity_date'])
loans_df['reference_date'] = pd.to_datetime(loans_df['reference_date'])

# Find the min and max dates
min_date_accounts = accounts_df['reference_date'].min()
max_date_accounts = accounts_df['reference_date'].max()

min_date_deposits = deposits_df[['start_date', 'maturity_date', 'reference_date']].min().min()
max_date_deposits = deposits_df[['start_date', 'maturity_date', 'reference_date']].max().max()

min_date_loans = loans_df[['start_date', 'maturity_date', 'reference_date']].min().min()
max_date_loans = loans_df[['start_date', 'maturity_date', 'reference_date']].max().max()

# Overall min and max dates
overall_min_date = min(min_date_accounts, min_date_deposits, min_date_loans)
overall_max_date = max(max_date_accounts, max_date_deposits, max_date_loans)

print(f"Overall Min Date: {overall_min_date}")
print(f"Overall Max Date: {overall_max_date}")

Overall Min Date: 2019-12-01 00:00:00
Overall Max Date: 2035-12-05 00:00:00


In [118]:
import pandas as pd

# Define the date range for the date dimension based on overall min and max dates
date_range = pd.date_range(start='2019-12-01', end='2035-12-05')

# Create a DataFrame for the date dimension
date_dimension = pd.DataFrame(date_range, columns=['date'])
date_dimension['date_key'] = date_dimension['date'].dt.strftime('%Y%m%d').astype(int)
date_dimension['year'] = date_dimension['date'].dt.year
date_dimension['month'] = date_dimension['date'].dt.month
date_dimension['day'] = date_dimension['date'].dt.day
date_dimension['quarter'] = date_dimension['date'].dt.quarter
date_dimension['day_of_week'] = date_dimension['date'].dt.dayofweek
date_dimension['day_name'] = date_dimension['date'].dt.day_name()
date_dimension['month_name'] = date_dimension['date'].dt.month_name()
date_dimension['is_weekend'] = date_dimension['date'].dt.dayofweek >= 5

# Display the first few rows of the date dimension DataFrame
print(date_dimension.head())
# 

        date  date_key  year  month  day  quarter  day_of_week   day_name  \
0 2019-12-01  20191201  2019     12    1        4            6     Sunday   
1 2019-12-02  20191202  2019     12    2        4            0     Monday   
2 2019-12-03  20191203  2019     12    3        4            1    Tuesday   
3 2019-12-04  20191204  2019     12    4        4            2  Wednesday   
4 2019-12-05  20191205  2019     12    5        4            3   Thursday   

  month_name  is_weekend  
0   December        True  
1   December       False  
2   December       False  
3   December       False  
4   December       False  


### Loan_type_Dimension

In [119]:
loan_types = loans_df['loan_type'].unique()
loan_type_df = pd.DataFrame(loan_types, columns=['loan_type'])
loan_type_df['loan_type_key'] = range(1, len(loan_type_df) + 1)
loan_type_df

,loan_type,loan_type_key
0,Unsecured Personal Loan,1
1,Credit Card,2
2,Corporate Leasing,3


### Deposit_type_Dimension

In [120]:
deposit_types = deposits_df['deposit_type'].unique()
deposit_type_df = pd.DataFrame(deposit_types, columns=['deposit_type'])
deposit_type_df['deposit_type_key'] = range(1, len(deposit_type_df) + 1)
deposit_type_df

,deposit_type,deposit_type_key
0,Fixed,1


## Creating Fact Tables

In [121]:
fact_accounts = accounts_df.copy()
fact_deposits = deposits_df.copy()
fact_loans = loans_df.copy()

### Making Surrogate Primary Key

In [122]:
fact_accounts['account_surr_primarykey'] = range(1, len(fact_accounts) + 1)
fact_deposits['deposits_surr_primarykey'] = range(1, len(fact_deposits)+1)
fact_loans['loans_surr_primarykey'] = range(1, len(fact_loans)+1)


### Maping date_dimension to Fact_tables
1. Map all the dates with the key 
2. Drop the original columns

In [123]:
# Function to map dates to date keys
def map_date_to_key(df, date_column):
    df[date_column] = pd.to_datetime(df[date_column])
    df[date_column + '_key'] = df[date_column].dt.strftime('%Y%m%d').astype(int)
    return df


# Add date keys to fact_accounts
fact_accounts = map_date_to_key(fact_accounts, 'reference_date')

# Add date keys to fact_deposits
fact_deposits = map_date_to_key(fact_deposits, 'start_date')
fact_deposits = map_date_to_key(fact_deposits, 'maturity_date')
fact_deposits = map_date_to_key(fact_deposits, 'reference_date')

# Add date keys to fact_loans
fact_loans = map_date_to_key(fact_loans, 'start_date')
fact_loans = map_date_to_key(fact_loans, 'maturity_date')
fact_loans = map_date_to_key(fact_loans, 'reference_date')





In [124]:
fact_deposits.drop(columns = ['start_date','maturity_date','reference_date'], inplace=True)
fact_loans.drop(columns = ['start_date','maturity_date','reference_date'], inplace=True)
fact_accounts.drop(columns=['reference_date'], inplace = True)

### Maping cutomer_dimension to fact_tables

In [125]:
def map_customer_to_key(df, unique_customers_df):
    df = df.merge(unique_customers_df[['customer', 'customer_key']], on='customer', how='left')
    # df.drop(columns=['customer'], inplace=True)
    return df

# Add customer_key to fact_deposits and drop the original customer column
fact_deposits = map_customer_to_key(fact_deposits, unique_customers_df)

# Add customer_key to fact_loans and drop the original customer column
fact_loans = map_customer_to_key(fact_loans, unique_customers_df)





In [126]:
fact_deposits.drop(columns=['customer','customer_type'], inplace=True)
fact_loans.drop(columns=['customer','customer_type'], inplace=True)


### Maping account_name dimensions to fact tables

In [127]:
def map_account_name_to_key(df, account_name_df):
    df = df.merge(account_name_df, on='account_name', how='left')

    return df


fact_accounts = map_account_name_to_key(fact_accounts, unique_account_names)

In [128]:
fact_accounts.drop(columns=['account_name'],inplace=True)

### Maping Currency dimension to  fact tables

In [129]:
def map_currency_to_key(df, unique_currencies_df):
    df = df.merge(unique_currencies_df, on='currency', how='left')
    return df


# Add currency_key to fact_deposits and drop the original currency column
fact_deposits = map_currency_to_key(fact_deposits, unique_currencies_df)

# Add currency_key to fact_loans and drop the original currency column
fact_loans = map_currency_to_key(fact_loans, unique_currencies_df)

In [130]:
fact_deposits.drop(columns=['currency'], inplace= True)
fact_loans.drop(columns=['currency'], inplace=True)


### Mapping Country dimension to Fact tables

In [131]:
def map_country_to_key(df, unique_countries_df):
    df = df.merge(unique_countries_df, on='country', how='left')
    return df



# Add country_key to fact_deposits and drop the original country column
fact_deposits = map_country_to_key(fact_deposits, unique_countries_df)

# Add country_key to fact_loans and drop the original country column
fact_loans = map_country_to_key(fact_loans, unique_countries_df)


In [132]:
fact_deposits.drop(columns=['country'], inplace= True)
fact_loans.drop(columns=['country'], inplace=True)

### Mapping Deposit_type dimensions to  Fact tables

In [133]:
def map_deposit_type_to_key(df, deposit_type_df):
    df = df.merge(deposit_type_df, on='deposit_type', how='left')
    return df


# Add deposit_type_key to fact_deposits and drop the original deposit_type column
fact_deposits = map_deposit_type_to_key(fact_deposits, deposit_type_df)

In [134]:
fact_deposits.drop(columns=['deposit_type'], inplace=True)

### Mapping Loan_type dimensions to Fact tables

In [135]:
def map_loan_type_to_key(df, loan_type_df):
    df = df.merge(loan_type_df, on='loan_type', how='left')
    return df



# Add loan_type_key to fact_loans and drop the original loan_type column
fact_loans = map_loan_type_to_key(fact_loans, loan_type_df)



In [136]:
fact_loans.drop(columns=['loan_type'], inplace = True)

# Load data to Data warehouse

For the purpose of this assignment I have uploaded the files in a folder called Data warehouse but I can also upload them on Google Big Query using python package


1. **Dimension Table** Add three columns 'start_date', 'end_date' and 'active_flag' for Slowly changing dimensions type 2 and then load the data into data warehouse

2. **Fact Tables**  Ensure surrogate Primary key for Fact tables exist and then load 

## Dimension Tables

In [137]:
import pandas as pd
from datetime import datetime
import os

# Function to add SCD2 columns
def add_scd2_columns(df):
    df['start_date'] = datetime.now()
    df['end_date'] = pd.to_datetime('2262-04-11')
    df['active_flag'] = 'Y'
    return df

os.makedirs('Data_warehouse', exist_ok=True)

# Add SCD2 columns to each dimension table
unique_customers_df = add_scd2_columns(unique_customers_df)
unique_countries_df = add_scd2_columns(unique_countries_df)
unique_currencies_df = add_scd2_columns(unique_currencies_df)
unique_account_names = add_scd2_columns(unique_account_names)
date_dimension = add_scd2_columns(date_dimension)
loan_type_df = add_scd2_columns(loan_type_df)
deposit_type_df = add_scd2_columns(deposit_type_df)



# Function to save DataFrame to CSV
def save_to_csv(df, file_name):
    df.to_csv(file_name, index=False)

# Save each updated DataFrame to CSV
save_to_csv(unique_customers_df, 'Data_warehouse/unique_customers.csv')
save_to_csv(unique_countries_df, 'Data_warehouse/unique_countries.csv')
save_to_csv(unique_currencies_df, 'Data_warehouse/unique_currencies.csv')
save_to_csv(unique_account_names, 'Data_warehouse/unique_account_names.csv')
save_to_csv(date_dimension, 'Data_warehouse/date_dimension.csv')
save_to_csv(loan_type_df, 'Data_warehouse/loan_type.csv')
save_to_csv(deposit_type_df, 'Data_warehouse/deposit_type.csv')

# The files are now ready to be loaded into the data warehouse


## Fact Tables

In [138]:

save_to_csv(fact_deposits, 'Data_warehouse/fact_deposits.csv')
save_to_csv(fact_loans, 'Data_warehouse/fact_loans.csv')
save_to_csv(fact_accounts, 'Data_warehouse/fact_accounts.csv')

## Persist the max surrogate keys for Fact tables

In [142]:
import pandas as pd
import os

# Create the Data_warehouse folder if it doesn't exist
os.makedirs('Data_warehouse', exist_ok=True)

max_deposits_key = fact_deposits['deposits_surr_primarykey'].max()
max_accounts_key = fact_accounts['account_surr_primarykey'].max()
max_loans_key = fact_loans['loans_surr_primarykey'].max()



max_keys_data = [
    {"table_name": "fact_deposits", "max_deposits_surr_primarykey": max_deposits_key},
    {"table_name": "fact_accounts", "max_account_surr_primarykey": max_accounts_key},
    {"table_name": "fact_loans", "max_surr_primarykey": max_loans_key}
]
max_keys_df = pd.DataFrame(max_keys_data)

max_keys_df['max_deposits_surr_primarykey'] = max_keys_df['max_deposits_surr_primarykey'].fillna(0).astype(int)
max_keys_df['max_account_surr_primarykey'] = max_keys_df['max_account_surr_primarykey'].fillna(0).astype(int)
max_keys_df['max_surr_primarykey'] = max_keys_df['max_surr_primarykey'].fillna(0).astype(int)

# Save the max keys data to a CSV file
max_keys_df.to_csv('Data_warehouse/max_keys.csv', index=False)

# print("Max keys data saved to Data_warehouse/max_surr_keys.csv")
max_keys_df

,table_name,max_deposits_surr_primarykey,max_account_surr_primarykey,max_surr_primarykey
0,fact_deposits,90,0,0
1,fact_accounts,0,16,0
2,fact_loans,0,0,300


## Google Big Query Data warehouse upload files

In [140]:
import os
import pandas as pd
from google.oauth2 import service_account

# Set the environment variable for authentication
key_path = "C:/Users/epranei/Downloads/calm-cove-423918-t0-ce8d5f6922f1.json"
credentials = service_account.Credentials.from_service_account_file(key_path)

def save_to_bigquery(df, table_id):
    df.to_gbq(table_id, project_id=project_id, if_exists='replace', credentials=credentials)

# Define your project and dataset ID
project_id = 'calm-cove-423918-t0'
dataset_id = 'Advisense'

# DataFrames to be saved to BigQuery
dataframes = {
    'unique_customers': unique_customers_df,
    'unique_countries': unique_countries_df,
    'unique_currencies': unique_currencies_df,
    'unique_account_names': unique_account_names,
    'date_dimension': date_dimension,
    'loan_type': loan_type_df,
    'deposit_type': deposit_type_df,
    'fact_deposits': fact_deposits,
    'fact_loans': fact_loans,
    'fact_accounts': fact_accounts
}

# Save each DataFrame to a BigQuery table
for table_name, df in dataframes.items():
    table_id = f'{dataset_id}.{table_name}'
    save_to_bigquery(df, table_id)

# Save max_keys_df to BigQuery
max_keys_table_id = f'{dataset_id}.max_keys'
save_to_bigquery(max_keys_df, max_keys_table_id)


C:\Users\epranei\AppData\Local\Temp\ipykernel_20216\745117729.py:10: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(table_id, project_id=project_id, if_exists='replace', credentials=credentials)
C:\Users\epranei\AppData\Local\Temp\ipykernel_20216\745117729.py:10: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(table_id, project_id=project_id, if_exists='replace', credentials=credentials)
C:\Users\epranei\AppData\Local\Temp\ipykernel_20216\745117729.py:10: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(table_id, project_id=project_id, if_exist